In [1]:
import os
import sys
sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2018,11)

In [2]:
def power_level(x_val, y_val, serial_number):
    #x_val, y_val = position
    #Find the fuel cell's rack ID, which is its X coordinate plus 10.
    rack_id = x_val + 10
    #Begin with a power level of the rack ID times the Y coordinate.
    power_level = rack_id * y_val
    #Increase the power level by the value of the grid serial number (your puzzle input).
    power_level += serial_number
    #Set the power level to itself multiplied by the rack ID
    power_level *= rack_id
    #Keep only the hundreds digit of the power level (so 12345 becomes 3; numbers with no hundreds digit become 0).
    # strip to the left of hundreds digit
    power_level = power_level % 1000
    # zero to the right of hundreds digit
    power_level = power_level // 100
    #Subtract 5 from the power level.
    power_level -= 5
    return power_level

def grid_power_level(position, power_levels, grid_size=3):
    X=0
    Y=1
    grid = []
    for x_val in range(position[X], position[X] + grid_size):
        for y_val in range(position[Y], position[Y] + grid_size):
            grid.append((x_val,y_val))
    return sum([power_levels[point] for point in grid])

def calc_max_grid_power_level(start, end, power_levels, grid_size=3):
    grid_power_levels = {}
    max_power_level = 0
    max_top_left = None
    for x_val in range(start, end - grid_size):
        for y_val in range(start, end - grid_size):
            grid_power_levels[(x_val, y_val)] = grid_power_level((x_val, y_val), power_levels, grid_size)
            if grid_power_levels[(x_val, y_val)] > max_power_level:
                max_power_level = grid_power_levels[(x_val, y_val)]
                max_top_left = (x_val, y_val)
    return max_top_left, max_power_level
    
    

In [3]:
#Fuel cell at  3,5, grid serial number 8: power level 4.
#Fuel cell at  122,79, grid serial number 57: power level -5.
#Fuel cell at 217,196, grid serial number 39: power level  0.
#Fuel cell at 101,153, grid serial number 71: power level  4.

tests = {
    (3,5): 8,
    (122, 79): 57,
    (217, 196): 39,
    (101, 153): 71
}

print("Tesing power_level calcultaions")
for position, serial_number in tests.items():
    print(f"Fuel cell at  {position}, grid serial number {serial_number}: power level {power_level(position, serial_number)}.")

print()

serial_number = 18
serial_number = 42
serial_number = 3214
power_levels = {}
# Each fuel cell has a coordinate ranging from 1 to 300 in both the X (horizontal) and Y (vertical) direction.
# In X,Y notation, the top-left cell is 1,1, and the top-right cell is 300,1

for x_val in range(1, 301):
    for y_val in range(1, 301):
        power_levels[(x_val, y_val)] = power_level((x_val, y_val), serial_number)

max_top_left, max_power_level = calc_max_grid_power_level(1, 300, power_levels, 3)
print(f"Part 1: Max grid power is {max_power_level} from {max_top_left}")

# below was to debug grid calculation issues
# issue was pretty straight forward, I left off the +1 and +2 in the third
# tuple for rows 2 and 3
#for point in [(21, 15), (33, 45)]:
#    (x_val, y_val) = point
#    grid = (
#        ((x_val, y_val),     (x_val + 1, y_val),     (x_val + 2, y_val)),
#        ((x_val, y_val + 1), (x_val + 1, y_val + 1), (x_val + 2, y_val + 1)),
#        ((x_val, y_val + 2), (x_val + 1, y_val + 2), (x_val + 2, y_val + 2))
#    )
#    print(serial_number)
#    for row in grid:
#        print(row)
#    for row in grid:
#        for point in row:
#            print(f"{power_levels[point]:3} ", end="")
#        print()
#    print()






Tesing power_level calcultaions


TypeError: power_level() missing 1 required positional argument: 'serial_number'

In [ ]:
max_grid_power_level_value = 0
max_grid_id = None
if False:
    for serial_number in  [18, 42]:
        power_levels = {}
        for x_val in range(1, 301):
            for y_val in range(1, 301):
                power_levels[(x_val, y_val)] = power_level((x_val, y_val), serial_number)
        for grid_size in range(1, 301):
            for x_val in range(1, 301 - grid_size + 1):
                for y_val in range(1, 301 - grid_size + 1):
                    max_top_left, max_power_level = calc_max_grid_power_level(1, 300, power_levels, grid_size)
                    if max_power_level > max_grid_power_level_value:
                        max_grid_power_level_value = max_power_level
                        max_grid_id = tuple([*max_top_left,grid_size])
                        print(max_grid_power_level_value, max_grid_id)
        print(f"Part 2: Serial number: {serial_number}, Max grid power is {max_grid_power_level_value} from {max_grid_id}")

        


In [17]:
import numpy
from functools import partial

for serial_number in [3214]:
    # partial function to pass serial numver to power_level
    partial_func = partial(power_level, serial_number=serial_number)
    # build grid of powerlevels using numpy.fromfunction and our partial function
    grid = numpy.fromfunction(partial_func, (300, 300))
    previous_max = -1 * float('infinity')
    for width in range(1, 300):
        windows = sum(grid[x:x-width+1 or None, y:y-width+1 or None] for x in range(width) for y in range(width))
        maximum = int(windows.max())
        location = numpy.where(windows == maximum)
        print(width, maximum, location[0][0], location[1][0]+1)
        # sample data maximum only had one peak
        # with my input, it had two peaks, so allowing
        # for a greater drop before breaking
        if maximum < previous_max - 100:
            print(width, maximum, location[0][0], location[1][0]+1)
            break
        if maximum > previous_max:  
            result = (maximum,location[0][0], location[1][0], width)
            previous_max = maximum
    print(','.join(str(num) for num in result[1:]))

1 4 0 9
2 16 21 172
3 32 21 43
4 38 233 18
5 55 241 92
6 56 241 251
7 63 241 199
8 70 241 89
9 77 233 215
10 82 233 215
11 83 232 213
12 82 231 213
13 85 230 213
14 74 229 212
15 81 233 88
16 74 232 88
17 66 231 87
18 65 230 86
19 52 230 85
20 29 230 85
21 14 225 77
22 50 241 79
23 25 241 79
24 -9 241 78
25 -13 241 79
26 -33 241 79
26 -33 241 79
230,212,13
